Task 2

Importing all the necessary libraries

In [3]:
!pip install rasterio
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 69.8 MB/s eta 0:00:00


Mounting the Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Checking the content of the vector file

In [8]:
# Load the GeoJSON file to check its content
geojson_path = '/content/drive/MyDrive/GIS5120/lab-6-task-data/grid.geojson'

# Read the GeoJSON file
geojson_data = gpd.read_file(geojson_path)

# Display the first few rows to inspect the content
geojson_data.head()


,id,left,top,right,bottom,row_index,col_index,geometry
0,41,583445.254038,4258815.0,594992.259422,4248815.0,16,1,"POLYGON ((583445.254 4253815, 586332.005 42588..."
1,42,583445.254038,4248815.0,594992.259422,4238815.0,17,1,"POLYGON ((583445.254 4243815, 586332.005 42488..."
2,61,592105.508076,4303815.0,603652.513459,4293815.0,12,2,"POLYGON ((592105.508 4298815, 594992.259 43038..."
3,62,592105.508076,4293815.0,603652.513459,4283815.0,13,2,"POLYGON ((592105.508 4288815, 594992.259 42938..."
4,63,592105.508076,4283815.0,603652.513459,4273815.0,14,2,"POLYGON ((592105.508 4278815, 594992.259 42838..."


Function to calculate mean and standard deviation from the polygon features and the raster dataset

In [20]:
def zonal_statistics(raster_path, vector_path):
    """
    Calculates mean and standard deviation of raster values for each polygon in the vector dataset.

    Parameters:
        raster_path (str): Path to the single-band raster file.
        vector_path (str): Path to the shapefile or GeoJSON file.

    Returns:
        pd.DataFrame: DataFrame with unique polygon ID, mean, and standard deviation of raster values.
    """
    # Read the vector data
    vector_data = gpd.read_file(vector_path)

    # Print CRS of vector data
    print("Vector CRS before reprojection:", vector_data.crs)

    # Read the raster data
    with rasterio.open(raster_path) as src:
        print("Raster CRS:", src.crs)

        # Check CRS match and reproject if necessary
        if vector_data.crs != src.crs:
            print("CRS mismatch detected. Reprojecting vector data to match raster CRS.")
            vector_data = vector_data.to_crs(src.crs)
        else:
            print("CRS match confirmed. Proceeding without reprojecting.")

        # Create lists to store results
        means = []
        std_devs = []
        unique_ids = []

        # Loop through each polygon
        for i, row in vector_data.iterrows():
            # print(f"Processing Polygon ID: {row['id']}")  # Debug statement
            try:
                # Mask the image based on polygon geometry
                out_image, out_transform = rasterio.mask.mask(
                    dataset=src,
                    shapes=[row['geometry']],
                    crop=True
                )
                # Flatten the masked array to exclude no-data values
                masked_arr = out_image[0, :, :]
                valid_data = masked_arr[masked_arr != src.nodata]

                # Calculate mean and standard deviation
                mean_val = valid_data.mean() if valid_data.size > 0 else np.nan
                std_val = valid_data.std() if valid_data.size > 0 else np.nan

                # Append results
                means.append(mean_val)
                std_devs.append(std_val)
                unique_ids.append(row['id'])  # Use 'id' column as unique identifier

            except ValueError:
                print(f"Polygon {row['id']} does not intersect raster.")
                means.append(np.nan)
                std_devs.append(np.nan)
                unique_ids.append(row['id'])

            except Exception as e:
                print(f"Error processing polygon {row['id']}: {e}")
                means.append(np.nan)
                std_devs.append(np.nan)
                unique_ids.append(row['id'])

    # Create a DataFrame for the results
    result_df = pd.DataFrame({
        'id': unique_ids,
        'Mean': means,
        'Std_Dev': std_devs
    })

    print("Zonal statistics calculated.")  # Debug statement
    return result_df

# Setting the vector and raster path
raster_path = "/content/drive/MyDrive/GIS5120/lab-6-task-data/ndvi.tif"
vector_path = "/content/drive/MyDrive/GIS5120/lab-6-task-data/grid.geojson"

# Run the function and display results
result = zonal_statistics(raster_path, vector_path)


Vector CRS before reprojection: EPSG:32615
Raster CRS: EPSG:32615
CRS match confirmed. Proceeding without reprojecting.
Zonal statistics calculated.


In [21]:
print(result)

      id      Mean   Std_Dev
0     41  0.405855  0.067818
1     42  0.406319  0.058588
2     61  0.424627  0.065736
3     62  0.417399  0.071759
4     63  0.363553  0.140118
..   ...       ...       ...
367  560  0.333018  0.111100
368  561  0.334814  0.114300
369  562  0.301963  0.115530
370  581  0.312016  0.097210
371  582  0.294459  0.103302

[372 rows x 3 columns]


Joining the output with the vector file to get the required information

In [23]:
vi_shp = pd.merge(
    geojson_data,
    result,
    left_on='id',
    right_on='id'
)

Printing the first few rows of joined dataframe

In [24]:
print(vi_shp.head())

   id           left        top          right     bottom  row_index  \
0  41  583445.254038  4258815.0  594992.259422  4248815.0         16   
1  42  583445.254038  4248815.0  594992.259422  4238815.0         17   
2  61  592105.508076  4303815.0  603652.513459  4293815.0         12   
3  62  592105.508076  4293815.0  603652.513459  4283815.0         13   
4  63  592105.508076  4283815.0  603652.513459  4273815.0         14   

   col_index                                           geometry      Mean  \
0          1  POLYGON ((583445.254 4253815, 586332.005 42588...  0.405855   
1          1  POLYGON ((583445.254 4243815, 586332.005 42488...  0.406319   
2          2  POLYGON ((592105.508 4298815, 594992.259 43038...  0.424627   
3          2  POLYGON ((592105.508 4288815, 594992.259 42938...  0.417399   
4          2  POLYGON ((592105.508 4278815, 594992.259 42838...  0.363553   

    Std_Dev  
0  0.067818  
1  0.058588  
2  0.065736  
3  0.071759  
4  0.140118  


Saving the new vector file

In [25]:
vi_shp.to_file('/content/drive/MyDrive/GIS5120/lab-6-task-data/finalvector.shp')